In [1]:
import os
import time
import json
import datetime
import zoneinfo

import requests

from dotenv import load_dotenv, find_dotenv

from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import (
    MessageTextContent,
    ListSortOrder,
    McpTool,
    MCPToolDefinition,
    RequiredMcpToolCall,
    SubmitToolApprovalAction,
    ToolApproval,
    CodeInterpreterTool,
    FunctionTool,
    ToolSet,
)

In [2]:
load_dotenv(override=True)

PROJECT_ENDPOINT=os.getenv("PROJECT_ENDPOINT")
AZURE_DEPLOYMENT_NAME=os.getenv("AZURE_DEPLOYMENT_NAME")

In [3]:
# AI Project Client を初期化
project_client = AIProjectClient(
    endpoint=PROJECT_ENDPOINT,
    credential=DefaultAzureCredential()
)

In [ ]:
def agent_run_outputs(thread_id, project_client, file_id_to_name=None, target_dir="./output_images"):
    """
    指定したスレッドIDのRun実行結果（テキスト・画像）をNotebook上に表示＆画像は保存。
    アノテーションがあれば [ファイル名] で置換して表示。
    file_id_to_name: {file_id: file_name} 形式の辞書
    """
    if file_id_to_name is None:
        file_id_to_name = {}
    messages = project_client.agents.messages.list(thread_id=thread_id, order=ListSortOrder.ASCENDING)
    os.makedirs(target_dir, exist_ok=True)

    for message in messages:
        # テキスト出力（アノテーション対応）
        if message.text_messages:
            for txt in message.text_messages:
                text_value = txt.text.value
                # アノテーションで引用部分を置換
                if hasattr(txt.text, "annotations"):
                    for annotation in txt.text.annotations:
                        fid = getattr(annotation.file_citation, "file_id", None)
                        citation = file_id_to_name.get(fid, fid)
                        # 空白を入れて見やすく
                        text_value = text_value.replace(annotation.text, f" [{citation}]")
                print(f"{message.role.upper()}: {text_value}")
        
        # 画像出力
        if hasattr(message, "image_contents"):
            for image_content in message.image_contents:
                file_id = image_content.image_file.file_id
                file_name = f"{file_id}_image_file.png"
                project_client.agents.files.save(
                    file_id=file_id,
                    file_name=file_name,
                    target_dir=target_dir
                )
                print(f"Saved image: {file_name}")
                display(Image(filename=f"{target_dir}/{file_name}"))


In [106]:
from azure.ai.agents.models import FilePurpose

# Define the path to the file to be uploaded
file_path = "./input_data/product_info_1.md"

# Upload the file
file = project_client.agents.files.upload_and_poll(file_path=file_path, purpose=FilePurpose.AGENTS)
print(f"Uploaded file, file ID: {file.id}")

# Create a vector store with the uploaded file
vector_store = project_client.agents.vector_stores.create_and_poll(file_ids=[file.id], name="my_vectorstore")
print(f"Created vector store, vector store ID: {vector_store.id}")

Uploaded file, file ID: assistant-MZQva6BbJ4ETXp23y8xRAf
Created vector store, vector store ID: vs_chjBOH4NsK3rUINq0VN5gJkN


In [107]:
while True:
    status = vector_store.status.value
    print(f"Vector store status: {status}")
    if status == "completed":
        break
    if status == "expired":
        raise RuntimeError("Vector store has expired.")
    if status == "in_progress":
        print("Vector store is still being created, waiting for completion...")
        time.sleep(3)

Vector store status: completed


In [91]:
from azure.ai.agents.models import FileSearchTool

file_search = FileSearchTool(vector_store_ids=[vector_store.id])

In [92]:
agent  = project_client.agents.create_agent(
    model=AZURE_DEPLOYMENT_NAME,
    name="agent ",
    instructions=(
        "You are a helpful agent and can search information from uploaded files"
    ),
    description=(
        "You are a helpful agent and can search information from uploaded files"    
    ),
    tools=file_search.definitions,  # Tools available to the agent
    tool_resources=file_search.resources,  # Resources for the tools
)
print(f"Created Agent. AGENT_ID: {agent.id}")


Created Agent. AGENT_ID: asst_uaqKOSLLZXjtilvgf7wY02vI


In [93]:
agent_dict = agent.as_dict()
print(json.dumps(agent_dict, indent=2, ensure_ascii=False))

{
  "id": "asst_uaqKOSLLZXjtilvgf7wY02vI",
  "object": "assistant",
  "created_at": 1753531908,
  "name": "agent ",
  "description": "You are a helpful agent and can search information from uploaded files",
  "model": "gpt-4.1",
  "instructions": "You are a helpful agent and can search information from uploaded files",
  "tools": [
    {
      "type": "file_search"
    }
  ],
  "top_p": 1.0,
  "temperature": 1.0,
  "tool_resources": {
    "file_search": {
      "vector_store_ids": [
        "vs_hJQ4yQtmwVJxXgTgcpXfXIXh"
      ]
    }
  },
  "metadata": {},
  "response_format": "auto"
}


In [94]:
# Thread の作成
thread = project_client.agents.threads.create()
print(f"Created Thread. THREAD_ID: {thread.id}")

Created Thread. THREAD_ID: thread_bPAYhcbQtjtQbLWlpsBZIpOy


In [95]:
# メッセージの追加
user_message = "TrailMaster X4 の価格（ドルと円）を教えてください。"

message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message,
)

print(f"Added Message. MESSAGE_ID: {message.id}")

Added Message. MESSAGE_ID: msg_AvEB20PGRO3dizshllSiSzjc


In [96]:
run = project_client.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id
)

if run.status == "failed":
    print(f"Run failed: {run.last_error}")
else:
    agent_run_outputs(thread.id, project_client)

USER: TrailMaster X4 の価格（ドルと円）を教えてください。
ASSISTANT: TrailMaster X4 の価格（ドルと円）について、該当ファイルを検索します。少々お待ちください。
ASSISTANT: TrailMaster X4 の価格は**250ドル**です。  
円での価格は為替レートによって異なりますが、例えば1ドル＝150円の場合、**37,500円**程度になります（250ドル × 150円 = 37,500円）。  
正確な円価格は購入時の為替レートをご確認ください。

【ファイル出典】TrailMaster X4 Tent, price $250【5:1†product_info_1.md】 [assistant-4rue4DJXYF2HH6arEFvgCj]


In [101]:
agent_run_outputs(thread.id, project_client)

USER: TrailMaster X4 の価格（ドルと円）を教えてください。
ASSISTANT: TrailMaster X4 の価格（ドルと円）について、該当ファイルを検索します。少々お待ちください。
ASSISTANT: TrailMaster X4 の価格は**250ドル**です。  
円での価格は為替レートによって異なりますが、例えば1ドル＝150円の場合、**37,500円**程度になります（250ドル × 150円 = 37,500円）。  
正確な円価格は購入時の為替レートをご確認ください。

【ファイル出典】TrailMaster X4 Tent, price $250 [assistant-4rue4DJXYF2HH6arEFvgCj]


In [102]:
messages = project_client.agents.messages.list(thread_id=thread.id)
file_name = os.path.split(file_path)[-1]
for msg in messages:
    if msg.text_messages:
        last_text = msg.text_messages[-1].text.value
        for annotation in msg.text_messages[-1].text.annotations:
            citation = (
                file_name if annotation.file_citation.file_id == file.id else annotation.file_citation.file_id
            )
            last_text = last_text.replace(annotation.text, f" [{citation}]")
        print(f"{msg.role}: {last_text}")

MessageRole.AGENT: TrailMaster X4 の価格は**250ドル**です。  
円での価格は為替レートによって異なりますが、例えば1ドル＝150円の場合、**37,500円**程度になります（250ドル × 150円 = 37,500円）。  
正確な円価格は購入時の為替レートをご確認ください。

【ファイル出典】TrailMaster X4 Tent, price $250 [product_info_1.md]
MessageRole.AGENT: TrailMaster X4 の価格（ドルと円）について、該当ファイルを検索します。少々お待ちください。
MessageRole.USER: TrailMaster X4 の価格（ドルと円）を教えてください。
